In [ ]:
import os
import sys
import copy
import wget
import time
import asyncio
import warnings
import logging
from astropy import time as astropytime

import astropy.units as u
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle

import numpy as np

from lsst.ts import salobj
from lsst.ts.standardscripts.auxtel.attcs import ATTCS

%matplotlib inline

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
from lsst.ts.standardscripts.auxtel.attcs import parallactic_angle

In [ ]:
from astroquery.simbad import Simbad

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
attcs = ATTCS()

In [ ]:
await attcs.start_task

In [ ]:
lst = Angle(13.24, unit=u.hour)

In [ ]:
object_table = Simbad.query_object("HD116351")

In [ ]:
print(object_table)

In [ ]:
radec_icrs = ICRS(Angle(object_table["RA"][0], unit=u.hour),
                          Angle(object_table["DEC"][0], unit=u.deg))

In [ ]:
radec_icrs.ra.hour

In [ ]:
parallactic_angle(attcs.location, radec_icrs.ra, radec_icrs).deg

In [ ]:
lst = np.linspace(radec_icrs.ra.hour-6, radec_icrs.ra.hour+6, 100)
pa = np.zeros(100)

In [ ]:
for i in range(len(lst)):
    pa[i] = parallactic_angle(attcs.location, Angle(lst[i], unit=u.hour), radec_icrs).deg

In [ ]:
plt.plot(lst, pa)

In [12]:
# import lsst.afw.cameraGeom.utils as cameraGeomUtils
# import lsst.afw.display as afwDisplay
import lsst.daf.persistence as dafPersist
from lsst.ip.isr import AssembleCcdTask
from lsst.ip.isr.isrTask import IsrTask

In [ ]:
os.environ['FIREFLY_HTML'] = "slate.html"
os.environ['FIREFLY_URL'] = 'http://139.229.170.210:8080/firefly'

In [3]:
dataPath = '/mnt/dmcs/oods_butler_repo/repo/'
butler = dafPersist.Butler(dataPath)

In [20]:
dataId = dict(expId=2020021900319)

raw = butler.get('raw', dataId)

AT_O_20200219_000319: Dark time less than exposure time. Setting dark time to the exposure time.


In [15]:
data_ref = butler.dataRef('raw', **dataId)

In [17]:
isrConfig = IsrTask.ConfigClass()
isrConfig.doLinearize = False
isrConfig.doBias = True
isrConfig.doFlat = False
isrConfig.doDark = False
isrConfig.doFringe = False
isrConfig.doDefect = False
isrConfig.doAddDistortionModel = False
isrConfig.doSaturationInterpolation = False
isrConfig.doSaturation = False
isrConfig.doWrite = False

isrTask = IsrTask(config=isrConfig)

/opt/lsst/software/stack/python/miniconda3-4.7.10/envs/lsst-scipipe-4d7b902/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Config field doAddDistortionModel is deprecated: Camera geometry is incorporated when reading the raw files. This option no longer is used, and will be removed after v19.
  


In [21]:
exposure = isrTask.runDataRef(data_ref).exposure

AT_O_20200219_000302: Dark time less than exposure time. Setting dark time to the exposure time.


In [ ]:
display1 = afwDisplay.getDisplay(frame=1, host='139.229.170.210:8080', port=80, name='tiagos_display',
                                  url='http://139.229.170.210:8080/firefly')

In [ ]:
afwDisplay.setDefaultBackend('firefly')
# disp = afwDisplay.Display(5)
# disp.scale('linear','zscale')
display1.mtv(exposure)
# cameraGeomUtils.overlayCcdBoxes(raw.getDetector(), isTrimmed=True, display=disp)

In [1]:
import lsst.afw.display as afw_display
afw_display.setDefaultBackend('lsst.display.firefly')
display1 = afw_display.getDisplay(frame=1, host='139.229.170.210:8080', port=80, name='tiagos_display',
                                  url='http://139.229.170.210:8080/firefly')

In [22]:
display1.mtv(exposure)